In [ ]:
import jieba
import numpy as np 
import pandas as pd 

dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
app_name = dfapp['app_name'].values.tolist()

jieba.load_userdict(app_name)

app_info = dfapp['app_info'].values.tolist()

app_info_segs = [jieba.lcut(x) for x in app_info]

In [ ]:
import logging
import os
from gensim.models import word2vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#sentences = word2vec.LineSentence('./in_the_name_of_people_segment.txt') 

model = word2vec.Word2Vec(app_info_segs, hs=1,min_count=1,window=3,size=100)  

In [ ]:
model.wv.save_word2vec_format('appwordvec.txt',binary = False)

In [ ]:
model.wv.most_similar(['app'],topn = 20)

### FastText 尝试

In [174]:
import jieba
import numpy as np 
import pandas as pd 

dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
#去除名字一样的

dfapp = dfapp.drop_duplicates(subset=['app_name'])
dfapp.index = range(len(dfapp))
app_name = dfapp['app_name'].values.tolist()

jieba.load_userdict(app_name)

app_info = dfapp['app_info'].values.tolist()
app_info_segs = [jieba.lcut(x) for x in app_info]

**构建字典**

In [76]:
from gensim import corpora, models,similarities
from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(app_info_segs)

In [77]:
# remove stop words and words that appear only once
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once

In [78]:
# 去除特别高频和特别低频的词
dictionary.filter_extremes(no_below=5, no_above=0.8, keep_n=100000)
dictionary.compactify()  # remove gaps in id sequence after words that were removed

In [79]:
import numpy as np 
import pandas as pd 
dftoken = pd.DataFrame()

Srtoken = pd.Series({k:v for v,k in dictionary.token2id.items()})
Srfreq  =  pd.Series(dictionary.dfs)

dftoken['name'] = Srtoken
dftoken['freq'] = Srfreq

In [80]:
# 去除长度为1的token
dftoken = dftoken.loc[[len(x)>1 for x in dftoken['name']],:]

In [81]:
# 去除数字token
dftoken_filter = dftoken[~dftoken.name.str.match("(\.|[0-9]|\%)*$")]
dftoken_filter.index = range(len(dftoken_filter ))

In [82]:
word_freq = dict(zip(dftoken_filter['name'],dftoken_filter['freq']))

In [11]:
dfapp[0:10]

,app_name,app_info,app_cate,app_score,download_cnt
0,融360,融360 —— 您随身的金融专家为什么要使用【融360】？下述内容不吹牛。1. 解决【贷款】...,理财,4.717574,2.321756e+07
1,微信,1.可以发语音、文字消息、表情、图片、视频30M流量可以收发上千条语音，省电省流量2.朋友圈...,社交,3.533574,7.601503e+09
2,设置,一款精美的屏幕开关手机小部件应用程序，实时装扮你的桌面，喜欢的你还等什么，赶快来下载吧！ 【...,系统,3.000000,8.294300e+04
3,支付宝,【支付宝介绍】蚂蚁金服旗下的支付宝，是以每个人为中心，拥有全球用户9亿的生活服务平台。目前，...,理财,3.583377,9.763013e+08
4,信息,iPhone信息是iPhone风格的消息应用程序。酷像IOS的UI。非常简单易用，就像iph...,工具,2.769231,3.027110e+05
5,QQ,-----QQ•乐在沟通-----√服务超过90%的移动互联网用户√多人视频、文件多端互传，...,社交,3.431373,8.703708e+09
6,指南针,这是一款功能实用，操作简明，性能出众的智能指南针。 它抛去了浮华，洗尽了铅尘。 优化后的...,旅游,4.553153,2.268551e+06
7,手机淘宝,手机淘宝（Android版）是阿里巴巴专为Android手机用户推出的满足其生活消费和线上购...,购物,4.543010,2.653544e+09
8,云服务,乐同步全新改版，这是一个全新的云服务乐同步是一款安全的手机私人数据备份与管理平台一键备份/恢...,安全,3.845070,3.067698e+07
9,WiFi万能钥匙,免费安全WiFi热点，无忧一键连接。无论身处何地，只要打开APP，看到分享热点，点击连接即可...,工具,2.681381,7.740153e+09


**训练向量**

In [39]:
dftoken_filter['name'].values.tolist()

['com',
 'http',
 '一天',
 '上市',
 '下述',
 '不停',
 '专家',
 '专属',
 '业务范围',
 '个人',
 '个人消费',
 '中国',
 '为什么',
 '产品',
 '亿美元',
 '介绍',
 '企业',
 '优势',
 '优惠',
 '优质',
 '估值',
 '低至',
 '使用',
 '便捷',
 '信用卡',
 '信赖',
 '俱乐部',
 '值得',
 '免费',
 '全面',
 '全面解决',
 '公众',
 '公司',
 '内容',
 '出行',
 '划算',
 '利率',
 '办卡',
 '即可',
 '吃喝玩乐',
 '各种',
 '各行',
 '吹牛',
 '垂直',
 '大全',
 '奉献',
 '学习',
 '安全',
 '完成',
 '官方',
 '官方网站',
 '实时',
 '客服',
 '小白',
 '平台',
 '年化',
 '微信',
 '微小',
 '成立',
 '我们',
 '技巧',
 '推荐',
 '提供',
 '搜索',
 '放款',
 '政策',
 '效率',
 '旅游',
 '旗下',
 '暖心',
 '服务',
 '期限',
 '此外',
 '步入',
 '消费者',
 '涵盖',
 '深知',
 '特点',
 '独角兽',
 '理应',
 '用户',
 '申请',
 '电话',
 '目前',
 '真情',
 '科技',
 '纽交所',
 '经济',
 '经营',
 '美国',
 '联系',
 '致力于',
 '融360',
 '融资',
 '装修',
 '解决',
 '论坛',
 '贷款',
 '费率',
 '资讯',
 '车房',
 '达人',
 '还款',
 '进修',
 '金融',
 '金融信息',
 '随身',
 '难题',
 'CD',
 '一起',
 '不再',
 '世界',
 '中文',
 '产生',
 '关注',
 '分享',
 '千条',
 '可以',
 '商品',
 '商店',
 '图书',
 '图片',
 '好玩',
 '封面',
 '帐号',
 '扫一扫',
 '扫描',
 '提醒',
 '摇一摇',
 '收发',
 '文字',
 '新闻',
 '明星',
 '有趣',
 '朋友',
 '朋友圈',
 '条码',
 '查看',
 '流量',

In [69]:
from gensim.test.utils import common_texts
from gensim.models import FastText

In [83]:
model = FastText(size=100, window=5, min_count=1, iter=10)

In [84]:
model.build_vocab([dftoken_filter['name'].values.tolist()])

In [85]:
model.train(app_info_segs,total_examples=model.corpus_count,epochs=model.epochs)

In [86]:
model.wv.most_similar('支付宝')


[('帮助', 0.6749873757362366),
 ('我们', 0.6736187934875488),
 ('打造', 0.6645970344543457),
 ('专业', 0.6633002161979675),
 ('便捷', 0.6582879424095154),
 ('平台', 0.6575096845626831),
 ('企业服务', 0.650421679019928),
 ('简单', 0.6501069068908691),
 ('随时随地', 0.6483274698257446),
 ('跟踪服务', 0.6479272842407227)]

**直接训练**

In [175]:
from gensim.models import FastText
model = FastText(app_info_segs, size=100, window=5, min_count=1, iter=10)

In [109]:
dfapp[0:40]

,app_name,app_info,app_cate,app_score,download_cnt
0,融360,融360 —— 您随身的金融专家为什么要使用【融360】？下述内容不吹牛。1. 解决【贷款】...,理财,4.717574,2.321756e+07
1,微信,1.可以发语音、文字消息、表情、图片、视频30M流量可以收发上千条语音，省电省流量2.朋友圈...,社交,3.533574,7.601503e+09
2,设置,一款精美的屏幕开关手机小部件应用程序，实时装扮你的桌面，喜欢的你还等什么，赶快来下载吧！ 【...,系统,3.000000,8.294300e+04
3,支付宝,【支付宝介绍】蚂蚁金服旗下的支付宝，是以每个人为中心，拥有全球用户9亿的生活服务平台。目前，...,理财,3.583377,9.763013e+08
4,信息,iPhone信息是iPhone风格的消息应用程序。酷像IOS的UI。非常简单易用，就像iph...,工具,2.769231,3.027110e+05
5,QQ,-----QQ•乐在沟通-----√服务超过90%的移动互联网用户√多人视频、文件多端互传，...,社交,3.431373,8.703708e+09
6,指南针,这是一款功能实用，操作简明，性能出众的智能指南针。 它抛去了浮华，洗尽了铅尘。 优化后的...,旅游,4.553153,2.268551e+06
7,手机淘宝,手机淘宝（Android版）是阿里巴巴专为Android手机用户推出的满足其生活消费和线上购...,购物,4.543010,2.653544e+09
8,云服务,乐同步全新改版，这是一个全新的云服务乐同步是一款安全的手机私人数据备份与管理平台一键备份/恢...,安全,3.845070,3.067698e+07
9,WiFi万能钥匙,免费安全WiFi热点，无忧一键连接。无论身处何地，只要打开APP，看到分享热点，点击连接即可...,工具,2.681381,7.740153e+09


In [162]:
# 查看最相似的词
model.wv.most_similar(u'付款')

[('代付款', 0.9624966382980347),
 ('付款单', 0.9480122923851013),
 ('付款吧', 0.9351937770843506),
 ('预付款', 0.9167335033416748),
 ('首付款', 0.909412145614624),
 ('支付款', 0.9085760712623596),
 ('支付现金', 0.8117930889129639),
 ('货到付款', 0.7752233743667603),
 ('支付', 0.7704386115074158),
 ('微支付', 0.765473484992981)]

In [ ]:
# 查看句子vector
model.wv[app_info[1]]

In [179]:
#for i in range(1)
model.wv.similarity(app_info[1],app_info[5])

0.9101988

In [178]:
#for i in range(1)
model.wv.similarity(app_info[1],app_info[3])

0.87001127

In [166]:
#for i in range(1)
model.wv.similarity(app_info[12],app_info[15])

0.94673944

In [177]:
#for i in range(1)
model.wv.similarity(app_info[14],app_info[24])

0.9479313

In [176]:
#for i in range(1)
model.wv.similarity(app_info[14],app_info[21])

0.7956311

In [180]:
app_vec = pd.DataFrame()
for i in range(len(dfapp)):
#for i in range(10):
    name = dfapp['app_name'][i]
    app_info = dfapp['app_info'][i]
    app_vec[name] = model.wv[app_info]

In [182]:
dfappvec = app_vec.T

In [183]:
dfappvec.to_csv('app_vec.csv',sep = '\t',encoding = 'utf-8')

In [126]:
len(app_vec)

22260

(22260, 5)

### 计算app聚类

In [184]:
from sklearn.cluster import KMeans

In [185]:
X = dfappvec.values

In [186]:
X.shape

(22260, 100)

In [265]:
clt = KMeans(n_clusters = 100)

In [266]:
app_clusters = clt.fit_predict(X)

In [267]:
len(app_clusters)

22260

In [331]:
dfgroup['cluster'] = ['cluster_'+str(i) for i in range(len(dfgroup))]

In [333]:
cluster = dfgroup.pop('cluster')

In [335]:
dfgroup.insert(0,'cluster',cluster )

In [337]:
dfgroup.to_csv('app_cluster.csv',sep = '\t',encoding = 'utf-8',index = None) 

In [ ]:
app_clusters

In [270]:
dfcluster = dfapp[['app_name']].copy()

In [271]:
dfcluster['app_cluster'] = app_clusters.tolist()

In [272]:
dfgroup = dfcluster.groupby('app_cluster').agg({'app_name':lambda l:','.join([x for x in l])})

In [273]:
dfgroup.columns = ['apps']

In [276]:
dfgroup.iloc[1,:].values.tolist()

['唯品会,售后服务,滴滴出行,中国工商银行,京东金融,平安金管家,中信银行,企业微信,点刷,管家服务,微云,招才猫直聘,征信查询,交易猫,一亩田,社会扶贫,盛钱包成长版,知鸟,汽车大全,今借到,信掌柜,农行掌上银行,美团骑手,视界,乐教乐学,115,国美,盒伙人,汽修宝,网易严选,喔刷快捷版,小米有品,南方航空,电e宝,淘车,蓝墨云班课,借条大师,信用星球,闪送,灯塔党建在线,消息盒子,中关村在线,大唐天下,云之家,闪电盒子,机猫,无限极中国,芒果银行,蓝薪卡,中国人保,绿叶商城,e钱庄,浙商银行,平安校园,平安天下通,移动助手,网贷之家,东莞农商银行,土豆泥,贵州统计发布,优课优信,泰行销,兰州银行,天眼查,闪租侠,拉卡拉商户通,崇法法律咨询,绿叶惠购,柠檬浏览器,男衣库,周润花,海豹商城,V网通,富宝袋,速米袋,升学e网通,平安财富宝,现金宝,微距,YY信用,车享家,互生,NGA玩家社区,思埠微商,壹学车,聚宝盆,阿里村菇,会过VIP,汇智能通,票根,到梦空间,嘀嗒出租司机,车行168,淘优乐,花米钱包,义乌购,有令,小胖钱包,兴店,八门神器,小蜜单车,马管家,北京实时公交,豌豆商城,福袋,雨露百事通,云企信,立刻出行,依米钱包,企业服务,规范查阅,汇博人才网,亿金钱包,搜房网,云分享,藏书馆,递推宝,晋商银行,泰康医生,锂市民,神州农服,有机转转,精准扶贫,一键反馈,常熟农商银行,众托帮,赏鱼袋,徽常有财,联想生活,汇养车,包头交警,闪聊,多彩宝,海词词典,瑞联盟,批批网服装女装批发,易订货,极速驾培,福州地铁码上行,花啦啦,掌中保,易物网,小凡,掌上体检,看看社保,服务中心,天机阁,万得股票,客户达开店,爱转,碧斯诺兰,小禾医助,多风的天气,格力商城,享看,城市一账通,球探推荐,91借钱,地震速报,央视新闻+,乘车易,人人快送,直脉,盛世链商,超G名片,物种起源,行者骑行,我要个性,泉州银行,小蜜淘,东方时尚,贷罗盘,章鱼彩票,校趣多,一点公益,同城聊播,鸭题库,天天盈球,恒泰金玉管家,文明中国,一览,贷款大师,瑞升达,华安智赢,达令家,云种养,国民村镇银行,威海公交,嘉宝生活家,汇贷客,律律律师法律咨询,爱尚养猪,宝驾出行,荣耀畅玩手环,衣邦人,小牛牛,果宝三国,信达同花顺,幸福路,会员宝商户,现货宝,课栈,e乐充公交卡,杭银直销,志愿者打卡器,金字招牌

In [277]:
dfgroup.iloc[0,:].values.tolist()

['设备管理,百度网盘,赶集网,脉脉,悬浮菜单,极简桌面,百度文库,Biu神器,小伴龙,现金在线,淘车二手车,微博极速版,全国违章查询,豆果美食,猫眼,遥控精灵,中华万年历,美团开店宝,金山词霸,飞贷,优健康,零购官网,5173游戏交易,轻松筹,好游快爆,菱菱邦,小葱钱包,ECO,速钱,大大红包,米赚-手机赚钱,小腰包,e城e家,澳客彩票,晋江小说阅读,纷享销客,新浪博客,汽车在线,完美校园,么么钱包,钱咖,京东股票,网贷天眼,家长帮,京东掌柜宝,零零期分期小额贷款,福建交通罚没,分七云,酷划锁屏-手机赚钱,黑岩阅读,手机腾讯网,知识星球,康康在线,理论培训,保险助手,花间,IUNI备忘录,可可宝贝,微商魔库,美丽约,语音翻译官,考拉信用,小伴龙儿歌,手机写小说,微销通,喜购,豆瓣阅读,装修图库,按键精灵,继续教育,i云保,58商家通,中信建投证券,一起买买买,拍来贷,艺术签名设计专业版,无忧课堂,我和你,兼客兼职,智慧树教师版,和讯财经,向上网,地震预警,现金来了,跟谁学,孕期伴侣,i代,网易大神,易公交,吉工家,WPS便签,云上智农,信用帮,黄历万年历,衣二三,来艾,医学教育网,营销管理系统,欢乐KTV,蚂蚁微贷,聊聊,小勾学习圈,网易红彩,DT加油,蜜蜂钱包,无忧行,东吴秀财,1步单车,花源,豆瓣FM,猿辅导,西南金点子,恒易融,财通证券,砖题库公务员,抹茶美妆,齐鲁人才,多融理财,进件宝,天气万年历,优时贷,山航掌尚飞,位置修改器,宝宝学英语,啡哈健身,钱方好近商户,移动OA,模拟炒股,虎课,大宜宾,九库阅读,菁优网,东兴198,小牛借呗,北海365,中国国家地理,智通人才,友途车服,V聊,模拟卡车2014,牛牛汽车,天弘爱理财,综合素质平台,爱善天使,五十音图,通付宝,汽车报价之家,东海通,吉屋,启蒙听听儿歌故事,新媒之家,UtoVR,手机搜狐,巧虎之家,妈妈网轻聊,神兽金刚2天神地兽,笑神来了,每日经济新闻,car2go,早晚听课文,脱单大师,临床指南,妙优车,掌门1对1辅导,橙牛汽车管家,一键投影,话本小说,火车票查询购票,爱代驾司机端,趣拿,麦子赚,南方Plus,执业医师真题,今日校园,省心宝汽车,库拍,句读,必胜公考,MBA智库,妈妈100,爱车在线,师生通新版,爱今天,溧阳论坛,股票雷达,小强停车,股票入门,朗朗点读,外勤助手,装修管家,口袋操盘

In [ ]:
### dfgroup.iloc[22,:].values.tolist()

In [328]:
dfgroup.iloc[33,:].values.tolist()

['便签,延时摄影,华为钱包,华为游戏中心,隐私空间,美拍,荣耀阅读,影音先锋,潮自拍,咪咕阅读,三星音乐,Keep,兴趣部落,CAD快速看图,咪咕音乐,咪咕爱看,同桌游戏,快看漫画,乐看搜索,OneNote,小天才,手机电视,滚动的天空,奥维互动地图,六合宝典,咪咕灵犀,迅游手游加速器,美甲帮,智能证件照,小米锁屏画报,铂涛旅行,橙子VR,桌面日历,贪吃蛇大战,绝地求生手游助手,图片合成器,体检宝测血压视力心率,贝乐虎儿歌,多玩我的世界盒子,VIP浏览器,随手便签,易互动,号码百事通,必要,快读全本小说,赢彩彩票,巧影,爱动漫,Cutie,美丽修行,百度VR,小米应用商店,谁是卧底,C5GAME,吉祥一号,欧朋浏览器,金机宝,钱夹子旅行,MixV,网易游戏论坛,CIT,叶罗丽精灵梦,僵尸榨汁机,朋友手机定位,任我花,很皮语音包,刷机精灵,怕怕,八分音符酱,豪门足球风云,Compass,尤果圈,SNKRS,好奇心日报,魔法公主礼仪学院,秀壁纸,多牛百变方块,快头条,Camera,电视果,截图神器,超级导购,封神召唤师,Anyview阅读,汉字转拼音,转机宝,埋堆堆,贪婪洞窟,暴走表情大全,寻欢,光影报名照,连环夺宝,易通商城,国民麻将,简单解压RAR,彩票33,唯乐,皮皮影视,Ticwear助手,叽里呱啦,百词斩爱阅读,柳州银行,铃声大全-官方版,纹字锁屏,萌新记账,图解电影,乐Max,306彩票,爱相机,窝趣美甲,先游,品值,兔兔儿歌,吉吉影音,Seed,谷壳宝,喵特,爱听听书,盖得排行,51CTO学院,Spotify,X浏览器,我去彩票站,POCO摄影,潮汐,7mall,梦之岛之旅,星空视频壁纸,Gallery,超级触控,Extreme Car Driving Simulator,随身炉石传说,工程随手拍,Cinepic,兜兜公交,飞传,一闪,凤凰娱乐,结婚小秘书,Rainbow,催眠大师,HOT男人,狗狗助手,派派红包,番茄ToDo,海豚动态壁纸,搞笑斗图大师,酷听听书,卧虎藏龙贰,中至南昌麻将,么么红包,吉林行,西瓜影音播放器,妖神记,幸福绿城,猫扑小说,儿歌大全,FilmoraGo,HelloTalk,钢琴块2（别踩白块儿2）,葫芦侠我的世界,钱包信用,遥控专家酷控,KMPlayer,减肥小秘书,百家号,充电助手,斐讯运动,科技侠,糖果苏打传奇,神剪手,朗

In [212]:
dfgroup.iloc[3,:].values.tolist()

['今日头条,一点资讯,51信用卡管家,宜人贷借款,人人贷借款,钱站,新闻资讯,快钱钱包,功夫贷,魔借,好钱包,读秒钱包,多享金汇,拿下钱包,东方头条,国务院,1905电影网,毛豆新车,首汽约车,周转王,超级视频,信用白卡,松鼠贷,好豆,酒仙网,爱贷借款,天下信用,7贷分期,安吉星,阳光安心贷,闲来跑得快,好分数,石榴直播,有利网,阿里众包,期待合伙人,小熊油耗,宗易汇,客运12308,小牛在线,WANTS好物,凤凰金融,信用金库,人人爱家金融,汇通财经,青岛公交查询,大象就医,金色财经,荔枝闪贷,真融宝,蚂蚁小微贷,钱牛牛,优信新车,宜贷网,钱盆网,化龙巷,应届生求职,小氢借钱,招财猫理财,小联钱袋,立借管家,洋钱罐,掌悦理财,起点直播,富勤金融,宏亚金融,付融宝,恋恋,钱罐子,小学同步课堂,一贯,银湖网,迷说,在行,谊入贷,爆米花视频,海象理财,蜂投借款,呆脸猫,乾包,bong,赚吧理财,挖链,交易虎,第一调查,轻易贷,米赚众测,芝麻小金,小金理财,市民网,美团拍店,融贝网,虾买商城,十八腔,派说,易融之家,零用金,万盈金融,向前金服,风车理财,钱时代理财,火星圈,护宝星,爱零工,良仓,联金所,哥哥热血捕鱼,长途拼车,钱粒账单,张飞充电,BD沃客,招商贷,前金融,拿下钱包Pro版,钱方商户,首金金融,海印金服,水滴保,6320优惠券,云阳人家,天使医生,合众e贷,今日借款,民投金服,小马金融,好东东商城,龙贷,百大易购,柚子理财,咔嚓保,钱香金融,e鹭理财,抗癌卫士,八条鱼理财,恒昌财富,黄金树,考一考,财富派,白菜金融,合众国金,废钢网,大麦理财,楚雄网,先锋浏览器,资产加,中国象棋棋谱,新滨海论坛,海星宝理财,钜宝盆,中国金融网,三明鱼网,快贷吧,生物谷,新青年麻醉论坛,汇鼎理财,东方理财师,随手控制中心,乡邻,商卡通会员管理,潮州音字典,帝象金融,首E家,石投金融,海淘网,掌盈金服,鹭岛金服,今日捷财,Bayram,财来贷款,阡陌,百狐网,X控:手机遥控器,新乐山,小狗钱钱,爱洛阳,初见理财,翡翠岛理财,新华金典理财,玉扳手,邦帮堂,钱到位,网贷天下,海上鲜,网投网,网上车市,仁仁分期,经文诗歌,囧羊理财,随州网,快速贷款APP,信贷通,兼职库,会计网,非常旺铺,约玩喵,bong Classic,爱建始网,聚龙理财,乐钱,霍邱论坛,宝宝学数数,贯通金服,贷

In [ ]:
dfgroup.to_csv('app_cluster',sep = '\t',encoding = 'utf-8')

In [213]:
dfgroup.iloc[4,:].values.tolist()

['百度地图,美图秀秀,平安口袋银行,天猫,B612咔叽,小米商城,智能家居,趣头条,凤凰新闻,球球大作战,下厨房,心悦俱乐部,淘票票,极简桌面,宜搜小说,Biu神器,WPS邮箱,二维码扫描,豆果美食,遥控精灵,还卡超人,众安保险,黄油相机,魔力相册,海报工厂,儿歌点点,新钱贷,共享单车,智能状态栏,球探体育比分,集享联盟,安兔兔评测,ETC车宝,网易手游管家,多看阅读,PicsArt,录音宝,nice,形色,GuitarTuna,91桌面,店长直聘,微爱,妈妈帮,堆糖,扇贝单词,游戏盒子,粉笔,京东阅读,啪啪游戏厅,悦跑圈,天天趣闻,酷街,米赚-手机赚钱,澳客彩票,小度掌柜,卷皮-9.9包邮,地铁通,快手看片,拼立得,火拼双扣,新氧美容,每天读点故事,爱唱,多锐运动,花酷,小宝钱袋,火辣健身,葡萄游戏厅,洋葱数学,冲浪导航,酷划锁屏-手机赚钱,随便走,黑岩阅读,叮咚钱包,智联卓聘,网信,九机网,洋码头,优美图,康康在线,8684公交,花间,赚客-手机赚钱,萌店,语音小秘书,QQ浏览器VR,鲨鱼记账,PonyCar,贝聊家长版,烘焙帮,沪江小D词典,平安好房,租我么,MONO,微销通,InShot,装修图库,按键精灵,扇贝口语,PINS,横店电影城,创业邦,礼物说,蓝信,花伴侣,我和你,非小号,追啊,智慧树教师版,超能界,月光蓝卡,百度汉语,向上网,日日煮,地震预警,书链,玩GO,猫耳FM,选股宝,孕期伴侣,最美闹钟,千变双扣,现贷猫,益盟操盘手加强版,凹凸租车,现金红包,微商管家,微话,省省回头车,好爸妈点读,次元社,汽车大师,微录客,车到加油,欢乐KTV,平安浙江,小鱼网,捷报比分,会说话的金杰猫,DT加油,土流网,bodivis,91贷款,活动行,双扣全集,凯励程,火星小说,魔音相册,坐公交,信用猫,考试宝典,奇妙昆虫世界,贝才,点购商城,建造师快题库,天天来塔防,厦航E鹭飞,聚币,12306生活,可米酷漫画,乐享四人斗地主,生活日历,达令全球好货,砖题库公务员,美美小店,证件照研究院,Owhat,护眼宝精简版,对面,财经头条,收趣,玉溪高古楼,防蹭网大师,爱抢购,六龙争霸3D,没钞蜂,位置修改器,小花秀VR直播,路由宝,綦江在线,BBC双语新闻,有谱么,大宜宾,耀莱成龙国际影城,国酒茅台,北海365,南方周末,短线王,甬派,神马专车,桔色,搜狐邮箱,萌宝儿歌

In [214]:
dfgroup.iloc[5,:].values.tolist()

['手机淘宝,下载管理器,媒体共享,互传,寻找美图手机,小米社区,企业QQ,船讯网,郑州银行,雄迈看看,昆仑银行,三星手机服务通,深圳航空,设备检测,维基百科,象棋巫师,经典蜘蛛纸牌,简易计算器,Google Play Store,小米便签,沃好商城,应用宝HD,智能连接,WiFi万能管理器,应用超市,雅堂金融,Freeme系统,Screen Recorder,万能ROOT,摩托罗拉迁移,长治银行,德众,媒体服务器,掌上教务,湖州银行移动银行,SmartWatch 2,WordPress,Climber,程序转移器,国开证券,中维安全管家,Glitchr,共创业,新蛋商城,银信富,中韩词典,任丘停车,3D平衡滚球']

### appcate

In [338]:
import numpy as np 
import pandas as pd 



In [344]:
dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
dfapp = dfapp.drop_duplicates(subset = ['app_name'])

In [345]:
dfcate = dfapp[['app_name','app_cate']].copy()

In [346]:
dfcategroup = dfcate.groupby('app_cate').agg({'app_name':lambda l:','.join([x for x in l])})

In [348]:
dfcategroup.insert(0,'app_cate',dfcategroup.index)

In [349]:
dfcategroup.insert(0,'cate_name')

,app_cate,app_name
app_cate,,
休闲益智,休闲益智,"珊瑚海,媒体存储器,贪吃蛇大作战,球球大作战,节奏大师,迷你世界,我的汤姆猫,腾讯桌球,我的..."
体育竞速,体育竞速,"登山赛车,天天飞车,城市飞车,街头篮球,FIFA足球世界,街机台球大师,疾风飞车世界,汤姆猫..."
健康,健康,"华为穿戴,三星健康,平安好医生,悦动圈,Keep,小米运动,咕咚,健康卫士,乐心运动,优健康..."
儿童,儿童,"儿童模式,宝宝树孕育,小伴龙,儿歌点点,西瓜皮,凯叔讲故事,贝乐虎儿歌,宝宝小警察,宝宝巴士..."
出行,出行,"高德地图,百度地图,驾驶模式,神州专车,位置服务,找回手机,微车违章查询,滴滴顺风车,奥维互..."
办公,办公,"WPS Office,设备管理,QQ邮箱,钉钉,脉脉,Microsoft Word,Micr..."
动作冒险,动作冒险,"Dark,汤姆猫跑酷,天天炫斗,大爆炸,滑雪大冒险,崩坏3,登山赛车2,三国战纪,奥特曼传奇..."
娱乐,娱乐,"游戏中心,西瓜视频,掌上英雄联盟,夜景,心悦俱乐部,TapTap,最右,快手直播伴侣,同桌游..."
安全,安全,"云服务,QQ安全中心,隐私空间,系统管家,百度手机卫士,360手机卫士,私密空间,隐私保护,..."


In [354]:
dfcategroup.to_csv('app_cate.csv',sep = '\t',encoding = 'utf-8',index = None )

In [355]:
a = {1,2,3}
b = {2,4,6}
a&b

{2}

In [352]:
dfcategroup 

,app_cate,app_name
app_cate,,
休闲益智,休闲益智,"珊瑚海,媒体存储器,贪吃蛇大作战,球球大作战,节奏大师,迷你世界,我的汤姆猫,腾讯桌球,我的..."
体育竞速,体育竞速,"登山赛车,天天飞车,城市飞车,街头篮球,FIFA足球世界,街机台球大师,疾风飞车世界,汤姆猫..."
健康,健康,"华为穿戴,三星健康,平安好医生,悦动圈,Keep,小米运动,咕咚,健康卫士,乐心运动,优健康..."
儿童,儿童,"儿童模式,宝宝树孕育,小伴龙,儿歌点点,西瓜皮,凯叔讲故事,贝乐虎儿歌,宝宝小警察,宝宝巴士..."
出行,出行,"高德地图,百度地图,驾驶模式,神州专车,位置服务,找回手机,微车违章查询,滴滴顺风车,奥维互..."
办公,办公,"WPS Office,设备管理,QQ邮箱,钉钉,脉脉,Microsoft Word,Micr..."
动作冒险,动作冒险,"Dark,汤姆猫跑酷,天天炫斗,大爆炸,滑雪大冒险,崩坏3,登山赛车2,三国战纪,奥特曼传奇..."
娱乐,娱乐,"游戏中心,西瓜视频,掌上英雄联盟,夜景,心悦俱乐部,TapTap,最右,快手直播伴侣,同桌游..."
安全,安全,"云服务,QQ安全中心,隐私空间,系统管家,百度手机卫士,360手机卫士,私密空间,隐私保护,..."


In [366]:
dfvec = pd.read_csv('app_vec.csv',sep = '\t',encoding = 'utf-8',index_col = 0)

In [371]:
dfvechit = dfvec.loc[{'融360','微信','haha'},:]

In [374]:
app_vec = dfvechit.mean() 

In [375]:
[str(i) for i in app_vec ]

['-0.1215169969946146',
 '0.2263110801577568',
 '-0.11403783038258553',
 '-0.10580507852137089',
 '-0.13848163932561874',
 '0.05902207177132368',
 '0.05116071691736578',
 '-0.009881746955215934',
 '-0.06932300329208374',
 '-0.010298439767211674',
 '0.009753916179761289',
 '0.18767999112606049',
 '-0.06781348027288914',
 '-0.06153331324458122',
 '-0.04013209044933319',
 '0.05536395125091076',
 '0.04585990123450757',
 '0.04752705432474613',
 '-0.08146653696894646',
 '-0.07301262579858303',
 '0.03726204624399543',
 '-0.017123333178460598',
 '-0.0944965835660696',
 '0.010475757764652371',
 '-0.08655692730098963',
 '-0.04632934741675854',
 '0.009341204538941387',
 '-0.07783494889736176',
 '0.04652390629053116',
 '-0.06871942337602377',
 '-0.0407713046297431',
 '-0.07732049003243446',
 '0.1377851739525795',
 '0.14040375128388405',
 '0.049070386216044426',
 '-0.07649123482406141',
 '0.02299698139540851',
 '0.04463003389537334',
 '-0.0025462694466114044',
 '0.03093546722084284',
 '0.0595366302

In [365]:
dfvechit.mean(axis = 0)

0    -0.121517
1     0.226311
2    -0.114038
3    -0.105805
4    -0.138482
5     0.059022
6     0.051161
7    -0.009882
8    -0.069323
9    -0.010298
10    0.009754
11    0.187680
12   -0.067813
13   -0.061533
14   -0.040132
15    0.055364
16    0.045860
17    0.047527
18   -0.081467
19   -0.073013
20    0.037262
21   -0.017123
22   -0.094497
23    0.010476
24   -0.086557
25   -0.046329
26    0.009341
27   -0.077835
28    0.046524
29   -0.068719
        ...   
70    0.106017
71   -0.065686
72    0.008813
73   -0.024000
74   -0.023962
75    0.059404
76    0.105298
77   -0.021284
78    0.002518
79    0.015434
80    0.157682
81    0.057240
82    0.204195
83   -0.123985
84    0.040492
85    0.056879
86    0.024126
87   -0.006140
88    0.083524
89    0.019428
90    0.046018
91    0.016171
92    0.097835
93   -0.250120
94    0.039964
95   -0.177414
96   -0.006075
97    0.034593
98    0.053127
99   -0.011964
Length: 100, dtype: float64